<a href="https://colab.research.google.com/github/aettikang/bigdata_analysis_basic/blob/main/%EA%B3%84%EC%B8%B5%EC%A0%81%EA%B5%B0%EC%A7%91%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 샘플 데이터와 Hierarchical Clustering

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


np.random.seed(2021)

## 1. Data

### 1.1 Sample data

이번 실습에서는 강의에서 예제로 보여드린 데이터를 이용해 진행합니다.

In [ ]:
data = np.array(
    [
        (1, 5),
        (2, 4),
        (4, 6),
        (4, 3),
        (5, 3),
    ]
)

In [ ]:
data.shape

(5, 2)

In [ ]:
data

array([[1, 5],
       [2, 4],
       [4, 6],
       [4, 3],
       [5, 3]])

## 2. Hierarchical Clustering

Hierarchical Clustering은 `sklearn.cluster` 의 `AgglomerativeClustering` 를 이용합니다.  
강의에서 배운 연결 법들은 4가지로 다음과 같습니다.
1. 최단 연결법
2. 최장 연결법
3. 평균 연결법
4. 중심 연결법

이를 적용하기 위해서는 `linkage` argument를 통해서 가능합니다.
- average
    - 평균 연결법
- complete
    - 최장 연결법
- single
    - 최단 연결법
- ward
    - 중심 연결법
   
기본 값은 `ward` 입니다.

### 2.1 학습

우선 강의에서 예제로 사용한 최단 연결법입니다.

In [ ]:
from sklearn.cluster import AgglomerativeClustering


single_cluster = AgglomerativeClustering(
    distance_threshold=0, n_clusters=None, linkage="single"
)

In [ ]:
single_cluster.fit(data)

### 2.2 Dendrogram

In [ ]:
from scipy.cluster.hierarchy import dendrogram


def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    dendrogram(linkage_matrix, **kwargs, labels=["A", "B", "C", "D", "E"])

실제로 강의에서 사용한 것 과 같은지 확인합니다.

In [ ]:
plt.title('Hierarchical Clustering Dendrogram with single linkage')
plot_dendrogram(single_cluster, truncate_mode='level', p=3)
plt.show()

### 2.3 여러 개의 클러스터

위에서 사용한 argument들은 데이터 전부를 묶는 방식의 Clustering 입니다.  
이제 n 개의 clustering으로 만들어 보겠습니다.

#### 2.3.1 2개의 클러스터

In [ ]:
single_cluster_2 = AgglomerativeClustering(
    n_clusters=2, linkage="single"
)

In [ ]:
single_cluster_2.fit(data)

AgglomerativeClustering(linkage='single')

In [ ]:
single_cluster_2.labels_

array([0, 0, 1, 0, 0])

In [ ]:
plt.figure(figsize=(7, 7))
plt.scatter(data[:, 0], data[:, 1], c=single_cluster_2.labels_)
for i, txt in enumerate(["A", "B", "C", "D", "E"]):
    plt.annotate(txt, (data[i, 0], data[i, 1]))

#### 2.3.2 3개의 클러스터

In [ ]:
single_cluster_3 = AgglomerativeClustering(
    n_clusters=3, linkage="single"
)

In [ ]:
single_cluster_3.fit(data)

In [ ]:
single_cluster_3.labels_

In [ ]:
plt.figure(figsize=(7, 7))
plt.scatter(data[:, 0], data[:, 1], c=single_cluster_3.labels_)
for i, txt in enumerate(["A", "B", "C", "D", "E"]):
    plt.annotate(txt, (data[i, 0], data[i, 1]))

## 3. 다른 연결법

이제는 다른 연결법들을 해보겠습니다.

### 3.1 평균 연결법

In [ ]:
avg_cluster = AgglomerativeClustering(
    distance_threshold=0, n_clusters=None, linkage="average"
)

In [ ]:
avg_cluster.fit(data)

In [ ]:
plt.title('Hierarchical Clustering Dendrogram with Average linkage')
plot_dendrogram(avg_cluster, truncate_mode='level', p=3)
plt.show()

### 3.2 최장 연결법

In [ ]:
max_cluster = AgglomerativeClustering(
    distance_threshold=0, n_clusters=None, linkage="complete"
)

In [ ]:
max_cluster.fit(data)

In [ ]:
plt.title('Hierarchical Clustering Dendrogram with Maximum linkage')
plot_dendrogram(max_cluster, truncate_mode='level', p=3)
plt.show()

### 3.3 중심 연결법

In [ ]:
centroid_cluster = AgglomerativeClustering(
    distance_threshold=0, n_clusters=None, linkage="ward"
)

In [ ]:
centroid_cluster.fit(data)

AgglomerativeClustering(distance_threshold=0, n_clusters=None)

In [ ]:
plt.title('Hierarchical Clustering Dendrogram with Centorid linkage')
plot_dendrogram(centroid_cluster, truncate_mode='level', p=3)
plt.show()

## 4. 마무리

In [ ]:
clusters = [
    ("Single", single_cluster),
    ("Average", avg_cluster),
    ("Maximum", max_cluster),
    ("Centroid", centroid_cluster),
]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 10))

for idx, (name, cluster) in enumerate(clusters):
    ax = axes[idx//2, idx%2]
    ax.set_title(f'Hierarchical Clustering Dendrogram with {name} linkage')
    plot_dendrogram(cluster, truncate_mode='level', p=3, ax=ax)